# Discovery das APIs do Instagram a partir do RapidApi

## Nós buscamos as seguintes informações para contas próprias e contas dos concorrentes


- Número de seguidores
  - Número total de seguidores para a conta
  - Data da busca 
- Publicações
  - Número total de publicações
  - IDs das publicações
  - Descrição das publicações
  - Foto/Link para a publicação
  - Data da busca
- Engajamento das publicações 
  - Definir quantas publicações são de cada tipo (Posts, Reels, Stories)
  - Para cada tipo de publicação econtrar:
    - Like
    - Comentários
    - Share

Obs:  
    
    Aparentemente o instagram limita para que todas APIs retornem apenas 12 posts por vez.
 
    Para buscar todos os posts precisaremos fazer inúmeras requisições.
 
    A partir das requisições teremos boas informações.

# Buscamos dados das seguintes marcas:

## Próprias: @
- mmartanoficial (MMartan)
- santistadecora (Santista)
- artex (Artex)

## Concorrentes: @
- artelasse (Artelasse)
- camicado (Camicado)
- casaalmeidaoficial (Casa Almeida)
- casariachuelo (Casa Riachuelo)
- karstenoficial (Karsten)
- mundodoenxoval (Mundo do Enxoval)
- trussardioficial (Trussardi)
- zeloloja (Zelo)

## APIs e endpoints:

Para buscar todos os dados talvez seja necessário o uso de mais de uma API para manter o plano gratuito.

Podemos modularizar os serviços para que usem serviços e endpoints diferentes.

Algumas APIs do RapidApi possuem retornos parecidos para endpoints de listagem de usuário ou postagens (aparentemente o limite para retornar apenas 12 postagens por requisição é padrão).

- API ATUAL https://rapidapi.com/social-api1-instagram/api/instagram-scraper-api2/playground/apiendpoint_b1301387-dc09-4b1f-ba39-b7b51d186b40

*Possíveis alternativas* 

- https://rapidapi.com/arraybobo/api/instagram-scraper-2022/playground/apiendpoint_4cede182-2b1d-4ade-a4c1-fc4c3577a01f
- https://rapidapi.com/mrngstar/api/instagram-api-20231/playground/apiendpoint_5113fc28-2703-4566-845d-3378a1f96bc7
- https://rapidapi.com/Instagapicom/api/instagram-scraper-20231/playground/apiendpoint_8a6bb604-8ced-4947-8560-50c221779c08
- 

## Objetos (DTO)

- DTOs para as contas e postagens

In [40]:
from dataclasses import dataclass
from typing import List, Optional, Union
import datetime

# ---------------------------------------------------------------------------------
@dataclass
class InstagramCaptionInfo:
    text: str
    created_at_utc: Union[datetime.date, str, None]
# ---------------------------------------------------------------------------------
    
# ---------------------------------------------------------------------------------
@dataclass
class InstagramMediaInfo:
    instagram_code: str # api -> code
    caption: InstagramCaptionInfo
    comment_count: Union[int, str]
    like_count: Union[int, str]
    media_name: str
    share_count: Union[int, str]
    taken_at: Union[datetime.date, str, None]
    is_video: Optional[bool] = False
    play_count: Optional[Union[int, str]] = None
    carousel_media_count: Optional[Union[int, str]] = None
# ---------------------------------------------------------------------------------

# ---------------------------------------------------------------------------------
@dataclass
class InstagramAccountInfo:
    name: str # api -> full_name
    username: str
    follower_count: Union[int, str]
    total_media: Union[int, str]
    profile_pic_url: str
    last_update: Union[datetime.date, str, None]
    instagram_user_id: Union[int, str] # api -> id
    media_info: Optional[List[InstagramMediaInfo]]
# ---------------------------------------------------------------------------------

- DTOs para as requisições

In [41]:
from dataclasses import dataclass
from typing import Dict, Optional

@dataclass
class IRequestInstagramParams:
    username: str
    base_url: str
    users_url: str
    posts_url: str
    x_rapidapi_key: str
    x_rapidapi_host: str
    user_querystring: str
    media_querystring: str
    pagination_token: Optional[str] = None


## Código úteis para modularização

- Services Enum
- Usuários Enum

In [44]:
from typing import Literal

ServiceNames = Literal[
        "instagram_scrapper_api"
    ]

TrackedUsers = Literal[
    "altenburg.oficial",
    "altenburghaus",
    "artelasse",
    "artex",
    "buddemeyeroficial",
    "casaalmeidaoficial",
    "casariachuelo",
    "casa.sonno",
    "hoomybr",
    "karstenoficial",
    "mmartanoficial",
    "santistadecora",
    "trussardioficial",
    "trousseauoficial",
    "zeloloja"
]

list_of_tracked_users = [
    "altenburg.oficial",
    "altenburghaus",
    "artelasse",
    "artex",
    "buddemeyeroficial",
    "casaalmeidaoficial",
    "casariachuelo",
    "casa.sonno",
    "hoomybr",
    "karstenoficial",
    "mmartanoficial",
    "santistadecora",
    "trussardioficial",
    "trousseauoficial",
    "zeloloja"
]

- Parametros necesários para a requisição

In [11]:
def get_service_params(service_name: ServiceNames, username: TrackedUsers) -> IRequestInstagramParams:
    services_availables = {
        "instagram_scrapper_api": IRequestInstagramParams(
            username=username,
            base_url="https://instagram-scraper-api2.p.rapidapi.com/",
            users_url="v1/info",
            posts_url="v1.2/posts",
            x_rapidapi_host="instagram-scraper-api2.p.rapidapi.com",
            x_rapidapi_key="6f36805577msh5e42867c3bd4692p12525ajsn80e3751c2d32",
            user_querystring="username_or_id_or_url",
            media_querystring="pagination_token",
            pagination_token=None
        )
    }
    
    return services_availables[service_name]

- Formatação dos parametros de requisição

In [14]:
def format_params_to_headers(params: IRequestInstagramParams) -> Dict[str, str]:
    return {
        "x-rapidapi-host": params.x_rapidapi_host,
        "x-rapidapi-key": params.x_rapidapi_key
    }

def format_user_querystring(params: IRequestInstagramParams) -> Dict[str, str]:
    return {
        params.user_querystring: params.username
    }

def format_media_querystring(params: IRequestInstagramParams) -> Dict[str, str]:
    return {
        params.user_querystring: params.username,
        params.media_querystring: params.pagination_token
    }

def format_users_url(params: IRequestInstagramParams) -> str:
    return params.base_url + params.users_url

def format_media_url(params: IRequestInstagramParams) -> str:
    return params.base_url + params.posts_url

In [15]:
service_params = get_service_params(service_name="instagram_scrapper_api", username="mmartanoficial")

headers = format_params_to_headers(service_params)

querystring_users = format_user_querystring(service_params)

querystring_media = format_media_querystring(service_params)

users_url = format_users_url(service_params)

media_url = format_media_url(service_params)

In [45]:
import requests
response = requests.get(users_url, headers=headers, params=querystring_users)

In [47]:
response.json()

{'data': {'about': None,
  'account_badges': [],
  'account_type': 2,
  'active_standalone_fundraisers': {'fundraisers': [], 'total_count': 0},
  'adjusted_banners_order': [],
  'ads_incentive_expiration_date': None,
  'ads_page_id': 224527784251940,
  'ads_page_name': 'mmartan',
  'bio_links': [{'icon_url': '',
    'image_url': '',
    'is_pinned': False,
    'is_verified': False,
    'link_id': 17989304845577369,
    'link_type': 'external',
    'lynx_url': 'https://l.instagram.com/?u=https%3A%2F%2Flinktr.ee%2Fmmartanoficial%3Ffbclid%3DPAZXh0bgNhZW0CMTEAAabjqFFo99GBD_iWecsqXGRzu_27GkGjJBXLgBu6Jxv1U3ummP1a3Lescgc_aem_StzwIofrKCd-q0KOEIop4Q&e=AT04wYtJCEoZKOIx58H2m3fTTOq09pX2k4bUh1K9dbtiWdM72aQXPkaGpeTjFwWRWk9iBmjxA5Wq5ScOrfbZK0oeYMw8tsXXpjvS8mI',
    'open_external_url_with_in_app_browser': True,
    'title': '',
    'url': 'https://linktr.ee/mmartanoficial'}],
  'biography': 'Desde 1980 criando histórias com a sua casa, seja no toque macio dos lençóis, banho aconchegante ou aromas inc

## Mock das respostas da API

- API = Instagram Scrapper API [Target](https://rapidapi.com/social-api1-instagram/api/instagram-scraper-api2/playground/apiendpoint_b1301387-dc09-4b1f-ba39-b7b51d186b40)

In [107]:
import json

def save_response_to_json(username: TrackedUsers, response):
    with open(username + '.json', 'a', encoding='utf-8') as f:
        json.dump(response, f, ensure_ascii=False, indent=4)

def load_response_from_json(username: TrackedUsers):
    with open(username + '.json', 'r', encoding='utf-8') as f:
        return json.load(f)


In [48]:
response.json()

{'data': {'about': None,
  'account_badges': [],
  'account_type': 2,
  'active_standalone_fundraisers': {'fundraisers': [], 'total_count': 0},
  'adjusted_banners_order': [],
  'ads_incentive_expiration_date': None,
  'ads_page_id': 224527784251940,
  'ads_page_name': 'mmartan',
  'bio_links': [{'icon_url': '',
    'image_url': '',
    'is_pinned': False,
    'is_verified': False,
    'link_id': 17989304845577369,
    'link_type': 'external',
    'lynx_url': 'https://l.instagram.com/?u=https%3A%2F%2Flinktr.ee%2Fmmartanoficial%3Ffbclid%3DPAZXh0bgNhZW0CMTEAAabjqFFo99GBD_iWecsqXGRzu_27GkGjJBXLgBu6Jxv1U3ummP1a3Lescgc_aem_StzwIofrKCd-q0KOEIop4Q&e=AT04wYtJCEoZKOIx58H2m3fTTOq09pX2k4bUh1K9dbtiWdM72aQXPkaGpeTjFwWRWk9iBmjxA5Wq5ScOrfbZK0oeYMw8tsXXpjvS8mI',
    'open_external_url_with_in_app_browser': True,
    'title': '',
    'url': 'https://linktr.ee/mmartanoficial'}],
  'biography': 'Desde 1980 criando histórias com a sua casa, seja no toque macio dos lençóis, banho aconchegante ou aromas inc

## Parsing das respostas da API para objetos

- Retirando apenas as informações necessárias a partir do json

In [21]:
mmartan_service_params = get_service_params(service_name="instagram_scrapper_api", username="mmartanoficial")

mmartan_response = load_response_from_json(mmartan_service_params)

In [128]:
import datetime

def parse_account_info(response):
    data = response['data']
    account_info = InstagramAccountInfo(
        name=data['full_name'],
        username=data['username'],
        follower_count=data['follower_count'],
        total_media=data['media_count'],
        last_update=datetime.datetime.now().isoformat(),
    )
    
    return account_info

In [129]:
info = parse_account_info(mmartan_response)
info

TypeError: 'Response' object is not subscriptable

In [53]:
import pandas as pd
import json

df = pd.read_json(json.dumps(mmartan_response), orient='index')
# response.json()
df

/var/folders/4t/13gtr7_95k58jpj2y37sh5nr0000gr/T/ipykernel_96592/2464827598.py:4: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(json.dumps(mmartan_response), orient='index')


,about,account_badges,account_category,account_type,active_standalone_fundraisers,additional_business_addresses,adjusted_banners_order,ads_incentive_expiration_date,ads_page_id,ads_page_name,...,show_shoppable_feed,spam_follower_setting_enabled,text_app_last_visited_time,third_party_downloads_enabled,total_ar_effects,total_igtv_videos,transparency_product_enabled,upcoming_events,username,whatsapp_number
data,NaN,[],,2,"{'fundraisers': [], 'total_count': 0}",[],[],NaN,224527784251940,mmartan,...,True,True,NaT,1,0,25,False,[],mmartanoficial,


In [55]:
data = {
    "name": [info.name],
    "username": [info.username],
    "follower_count": [info.follower_count],
    "total_media": [info.total_media],
    "profile_pic_url": [info.profile_pic_url],
    "last_update": [info.last_update],
    "instagram_user_id": [info.instagram_user_id],
    "media_info": [info.media_info]
}
df = pd.DataFrame(data)
df

,name,username,follower_count,total_media,profile_pic_url,last_update,instagram_user_id,media_info
0,mmartan,mmartanoficial,1172326,4618,https://scontent-waw2-1.cdninstagram.com/v/t51...,2024-10-01T10:32:01.462155,321202794,None


In [56]:
data = {
    "name": [],
    "username": [],
    "follower_count": [],
    "total_media": [],
    "profile_pic_url": [],
    "last_update": [],
    "instagram_user_id": [],
    "media_info": []
}
df_tracked_users = pd.DataFrame(data)
df_tracked_users

,name,username,follower_count,total_media,profile_pic_url,last_update,instagram_user_id,media_info


## Buscando dados e salvando

- primeiro vamos buscar todos os dados das marcas e salvar como json

In [75]:
import requests
import pandas as pd
import datetime

In [76]:
artelasse_service_params = get_service_params(service_name="instagram_scrapper_api", username="artelasse")
artex_service_params = get_service_params(service_name="instagram_scrapper_api", username="artex")
camicado_service_params = get_service_params(service_name="instagram_scrapper_api", username="camicado")
casa_almeida_service_params = get_service_params(service_name="instagram_scrapper_api", username="casaalmeidaoficial")
casa_riachuelo_service_params = get_service_params(service_name="instagram_scrapper_api", username="casariachuelo")
karsten_service_params = get_service_params(service_name="instagram_scrapper_api", username="karstenoficial")
mmartan_service_params = get_service_params(service_name="instagram_scrapper_api", username="mmartanoficial")
mundo_enxoval_service_params = get_service_params(service_name="instagram_scrapper_api", username="mundodoenxoval")
santista_service_params = get_service_params(service_name="instagram_scrapper_api", username="santistadecora")
trussardi_service_params = get_service_params(service_name="instagram_scrapper_api", username="trussardioficial")
zelo_service_params = get_service_params(service_name="instagram_scrapper_api", username="zeloloja")

In [61]:
headers = format_params_to_headers(artelasse_service_params)

In [91]:
artelasse_querystring_users = format_user_querystring(artelasse_service_params)
artelasse_querystring_media = format_media_querystring(artelasse_service_params)
artelasse_users_url = format_users_url(artelasse_service_params)
artelasse_media_url = format_media_url(artelasse_service_params)

artelasse_response = requests.get(artelasse_users_url, headers=headers, params=artelasse_querystring_users)

save_response_to_json(username="artelasse", response=artelasse_response.json())

TypeError: Object of type Response is not JSON serializable

In [93]:
artex_querystring_users = format_user_querystring(artex_service_params)
artex_querystring_media = format_media_querystring(artex_service_params)
artex_users_url = format_users_url(artex_service_params)
artex_media_url = format_media_url(artex_service_params)

artex_response = requests.get(artex_users_url, headers=headers, params=artex_querystring_users)

save_response_to_json(username="artex", response=artex_response.json())

In [94]:
camicado_querystring_users = format_user_querystring(camicado_service_params)
camicado_querystring_media = format_media_querystring(camicado_service_params)
camicado_users_url = format_users_url(camicado_service_params)
camicado_media_url = format_media_url(camicado_service_params)

camicado_response = requests.get(camicado_users_url, headers=headers, params=camicado_querystring_users)

save_response_to_json(username="camicado", response=camicado_response.json())

In [99]:
casa_almeida_querystring_media = format_media_querystring(casa_almeida_service_params)
casa_almeida_media_url = format_media_url(casa_almeida_service_params)
casa_almeida_querystring_users = format_user_querystring(casa_almeida_service_params)
casa_almeida_users_url = format_users_url(casa_almeida_service_params)

casa_almeida_response = requests.get(casa_almeida_users_url, headers=headers, params=casa_almeida_querystring_users)

save_response_to_json(username="casaalmeidaoficial", response=casa_almeida_response.json())

In [96]:
casa_riachuelo_querystring_users = format_user_querystring(casa_riachuelo_service_params)
casa_riachuelo_querystring_media = format_media_querystring(casa_riachuelo_service_params)
casa_riachuelo_users_url = format_users_url(casa_riachuelo_service_params)
casa_riachuelo_media_url = format_media_url(casa_riachuelo_service_params)

casa_riachuelo_response = requests.get(casa_riachuelo_users_url, headers=headers, params=casa_riachuelo_querystring_users)

save_response_to_json(username="casariachuelo", response=casa_riachuelo_response.json())

In [100]:
karsten_querystring_users = format_user_querystring(karsten_service_params)
karsten_querystring_media = format_media_querystring(karsten_service_params)
karsten_users_url = format_users_url(karsten_service_params)
karsten_media_url = format_media_url(karsten_service_params)

karsten_response = requests.get(karsten_users_url, headers=headers, params=karsten_querystring_users)

save_response_to_json(username="karstenoficial", response=karsten_response.json())

In [101]:
mmartan_querystring_users = format_user_querystring(mmartan_service_params)
mmartan_querystring_media = format_media_querystring(mmartan_service_params)
mmartan_users_url = format_users_url(mmartan_service_params)
mmartan_media_url = format_media_url(mmartan_service_params)

mmartan_response = requests.get(mmartan_users_url, headers=headers, params=mmartan_querystring_users)

save_response_to_json(username="mmartanoficial", response=mmartan_response.json())

In [102]:
mundo_enxoval_querystring_users = format_user_querystring(mundo_enxoval_service_params)
mundo_enxoval_querystring_media = format_media_querystring(mundo_enxoval_service_params)
mundo_enxoval_users_url = format_users_url(mundo_enxoval_service_params)
mundo_enxoval_media_url = format_media_url(mundo_enxoval_service_params)

mundo_enxoval_response = requests.get(mundo_enxoval_users_url, headers=headers, params=mundo_enxoval_querystring_users)

save_response_to_json(username="mundodoenxoval", response=casa_almeida_response.json())

In [103]:
santista_querystring_users = format_user_querystring(santista_service_params)
santista_querystring_media = format_media_querystring(santista_service_params)
santista_users_url = format_users_url(santista_service_params)
santista_media_url = format_media_url(santista_service_params)

santista_response = requests.get(santista_users_url, headers=headers, params=santista_querystring_users)

save_response_to_json(username="santistadecora", response=santista_response.json())

In [104]:
trussardi_querystring_users = format_user_querystring(trussardi_service_params)
trussardi_querystring_media = format_media_querystring(trussardi_service_params)
trussardi_users_url = format_users_url(trussardi_service_params)
trussardi_media_url = format_media_url(trussardi_service_params)

trussardi_response = requests.get(trussardi_users_url, headers=headers, params=trussardi_querystring_users)

save_response_to_json(username="trussardioficial", response=trussardi_response.json())

In [105]:
zelo_querystring_users = format_user_querystring(zelo_service_params)
zelo_querystring_media = format_media_querystring(zelo_service_params)
zelo_users_url = format_users_url(zelo_service_params)
zelo_media_url = format_media_url(zelo_service_params)

zelo_response = requests.get(zelo_users_url, headers=headers, params=zelo_querystring_users)

save_response_to_json(username="zeloloja", response=zelo_response.json())

- Montando um DataFrame a partir dos dados coletados

In [111]:
artelasse = parse_account_info(load_response_from_json('artelasse'))
artex = parse_account_info(load_response_from_json('artex'))
camicado = parse_account_info(load_response_from_json("camicado"))
casaalmeidaoficial = parse_account_info(load_response_from_json("casaalmeidaoficial"))
casariachuelo = parse_account_info(load_response_from_json("casariachuelo"))
karstenoficial = parse_account_info(load_response_from_json("karstenoficial"))
mmartanoficial = parse_account_info(load_response_from_json("mmartanoficial"))
mundodoenxoval = parse_account_info(load_response_from_json("mundodoenxoval"))
santistadecora = parse_account_info(load_response_from_json("santistadecora"))
trussardioficial = parse_account_info(load_response_from_json("trussardioficial"))
zeloloja = parse_account_info(load_response_from_json("zeloloja"))

In [127]:
from typing import List

users_list: List[InstagramAccountInfo] = [
    artelasse,
    artex,
    camicado,
    casaalmeidaoficial,
    casariachuelo,
    karstenoficial,
    mmartanoficial,
    mundodoenxoval,
    santistadecora,
    trussardioficial,
    zeloloja
]

data = {
    "name": [],
    "username": [],
    "follower_count": [],
    "total_media": [],
    "last_update": [],
}

df_tracked_users = pd.DataFrame(data)

for user in users_list:
    value = pd.Series([user.name, user.username, user.follower_count, user.total_media, user.last_update])
    df_tracked_users = df_tracked_users._append(dict(zip(df_tracked_users.columns, value)), ignore_index=True)

df_tracked_users


,name,username,follower_count,total_media,last_update
0,Artelassê,artelasse,283555.0,3894.0,2024-10-01T16:09:13.987954
1,ARTEX,artex,1029978.0,4081.0,2024-10-01T16:09:13.989424
2,Camicado,camicado,2450449.0,4474.0,2024-10-01T16:09:13.989649
3,Casa Almeida,casaalmeidaoficial,103733.0,1543.0,2024-10-01T16:09:13.989791
4,Casa Riachuelo,casariachuelo,1110401.0,2210.0,2024-10-01T16:09:13.989927
5,Karsten Oficial,karstenoficial,737112.0,4220.0,2024-10-01T16:09:13.990041
6,mmartan,mmartanoficial,1172320.0,4619.0,2024-10-01T16:09:13.990153
7,Casa Almeida,casaalmeidaoficial,103733.0,1543.0,2024-10-01T16:09:13.990262
8,Santista,santistadecora,1406067.0,4415.0,2024-10-01T16:09:13.990369
9,Trussardi,trussardioficial,294117.0,634.0,2024-10-01T16:09:13.990558


In [15]:
from commom.data_classes.instagram_data_class import SERVICE_NAME_LIST, IRequestInstagramParams, InstagramAccountInfo
fields = InstagramAccountInfo.__dataclass_fields__.keys()
for field_name in fields:
    print(field_name)

name
username
follower_count
total_media
last_update
biography
profile_pic


## InstagramService()

** Obs: O InstagramController deve seguir as mesmas funções e tratar a entrada **

Estrutura da classe
- init
- buscar as informações no instagram e salvar no banco
  - caso não exista nada no banco vamos criar um novo a partir de um dataframe
- buscar as informações no banco, transformar em ?excel? e enviar por email

In [5]:

from commom.instagram_data.instagram_data_manager import InstagramDataManager

# class InstagramService():
# init ->
data_manager: InstagramDataManager = InstagramDataManager()
# <-


In [13]:
import json

with open('./example_responses/artex.json', 'r', encoding='utf-8') as f:
    artex_response = json.load(f)
    
with open('./example_responses/mmartanoficial.json', 'r', encoding='utf-8') as f:
    mmartan_response = json.load(f)
    
with open('./example_responses/santistadecora.json', 'r', encoding='utf-8') as f:
    santista_response = json.load(f)

with open('./example_responses/artelasse.json', 'r', encoding='utf-8') as f:
    artelasse_response = json.load(f)

with open('./example_responses/karstenoficial.json', 'r', encoding='utf-8') as f:
    karsten_response = json.load(f)

with open('./example_responses/zeloloja.json', 'r', encoding='utf-8') as f:
    zelo_response = json.load(f)

with open('./example_responses/trussardioficial.json', 'r', encoding='utf-8') as f:
    trussardi_response = json.load(f)

In [14]:
from commom.instagram_data.instagram_data_formater import InstagramDataFormater
from commom.data_classes.instagram_data_class import InstagramAccountInfo
import json
from typing import List

# get_user_info -> PARTE 1
# -> fetch inicio
# data_manager.fetch_user_info(list_of_users=["artex", "mmartanoficial", "santistadecora"])
mocked_responses = [
    artex_response, 
    mmartan_response, 
    santista_response
]

data_manager_response: List[InstagramAccountInfo] = []

for mock in mocked_responses:
    r = InstagramDataFormater().parse_account_info(service_name="instagram_scrapper_api", response=mock)
    data_manager_response.append(r)

# -> fetch fim
# data_manager.fetch_user_info(list_of_users=["artex", "mmartanoficial", "santistadecora"])
# considerando que fizemos o fetch com sucesso deveremos ter data_manager_response
data_manager_response

[InstagramAccountInfo(name='ARTEX', username='artex', follower_count=1029978, total_media=4081, last_update='2024-10-03T12:17:41.552992', biography='Viva o conforto de ser você! ❤️\n• Use o cupom AXBEMVINDO10', profile_pic='https://instagram.fiev14-2.fna.fbcdn.net/v/t51.2885-19/455108042_860014102275169_1657998699154895928_n.jpg?stp=dst-jpg_s640x640&_nc_ht=instagram.fiev14-2.fna.fbcdn.net&_nc_cat=101&_nc_ohc=4eOdU047glIQ7kNvgEoWGCZ&_nc_gid=5cfade38bbff49fd81b817e66234acbe&edm=AEF8tYYBAAAA&ccb=7-5&oh=00_AYDdkTLoUcwe6gWLvC-b3BDrVTj14OrkFKakRc-7Tz44Aw&oe=67020201&_nc_sid=1e20d2'),
 InstagramAccountInfo(name='mmartan', username='mmartanoficial', follower_count=1172320, total_media=4619, last_update='2024-10-03T12:17:41.553015', biography='Desde 1980 criando histórias com a sua casa, seja no toque macio dos lençóis, banho aconchegante ou aromas inconfundíveis.', profile_pic='https://instagram.fiev22-2.fna.fbcdn.net/v/t51.2885-19/404293436_766960741933447_2962518092099297043_n.jpg?stp=dst-jp

In [20]:
# get_user_info -> PARTE 2
# -> load
current_dataset = data_manager.load_current_account_history_dataset()
current_dataset = data_manager.update_accounts_history_dataset(dataset=current_dataset, account_info_list = data_manager_response)

# get_user_info -> PARTE 3
# -> save
data_manager.save_current_account_history_dataset(dataset=current_dataset)

'\n            TO BE IMPLEMENTED\n        '

In [1]:
from commom.data_classes.instagram_data_class import TRACKED_ACCOUNT_LIST
from innovation_api.api.services.instagram_monitor_service import InstagramMonitorService

service = InstagramMonitorService()
tracked_users_list = TRACKED_ACCOUNT_LIST
usernames = [
    tracked_users_list[0],
    tracked_users_list[1],
    tracked_users_list[2],
]
usernames
response = service.update_accounts_info(usernames=usernames)
response

usernames ['altenburg.oficial', 'altenburghaus', 'artelasse']
info [InstagramAccountInfo(name='Bem dormir, bem viver', username='altenburg.oficial', follower_count=257544, total_media=3414, last_update='2024-10-03T14:05:50.453915', biography='Moda Casa com elegância, alta qualidade, tecnologias e práticas cada vez mais sustentáveis para o seu bem-estar.', profile_pic='https://scontent-mad1-1.cdninstagram.com/v/t51.2885-19/405204100_311861495005263_4564322676966450610_n.jpg?stp=dst-jpg_s640x640&_nc_ht=scontent-mad1-1.cdninstagram.com&_nc_cat=103&_nc_ohc=LtGaU_7eWFUQ7kNvgEGyk5Y&_nc_gid=e2aab7856cf44652b0ef97eb01e49c0a&edm=AO4kU9EBAAAA&ccb=7-5&oh=00_AYDHCuHFWpLnuhflzCY0LyAO0S9izGVS6U-37sVGh31kGQ&oe=6704A116&_nc_sid=164c1d'), InstagramAccountInfo(name='Altenburg Haus', username='altenburghaus', follower_count=25719, total_media=626, last_update='2024-10-03T14:05:50.875200', biography='Desde 1922, trazendo elegância e sofisticação atemporais para sua casa.', profile_pic='https://scontent-am

True

In [3]:
from commom.data_classes.instagram_data_class import TRACKED_ACCOUNT_LIST
from innovation_api.api.services.instagram_monitor_service import InstagramMonitorService

service = InstagramMonitorService()

df_user_info = service.data_manager.load_current_account_history_dataset()
df_user_info

,name,username,follower_count,total_media,last_update,biography,profile_pic
0,"Bem dormir, bem viver",altenburg.oficial,257544.0,3414.0,2024-10-03T14:05:50.453915,"Moda Casa com elegância, alta qualidade, tecno...",https://scontent-mad1-1.cdninstagram.com/v/t51...
1,Altenburg Haus,altenburghaus,25719.0,626.0,2024-10-03T14:05:50.875200,"Desde 1922, trazendo elegância e sofisticação ...",https://scontent-ams4-1.cdninstagram.com/v/t51...
2,Artelassê,artelasse,283629.0,3897.0,2024-10-03T14:05:51.360183,Enxovais premium de cama e banho.\nPara quem v...,https://scontent-mad2-1.cdninstagram.com/v/t51...


In [5]:
file = df_user_info.to_excel('user_info.xlsx', sheet_name='Seguidores e postagens',index=False)


In [1]:
from commom.data_classes.instagram_data_class import TRACKED_ACCOUNT_LIST
from innovation_api.api.services.instagram_monitor_service import InstagramMonitorService

service = InstagramMonitorService()

resp = service.send_report_xlsx(recipients=['joao.garcia@ammovarejo.com.br'])
resp

Seguidores_e_postagens.xlsx
                    name           username  follower_count  total_media  \
0  Bem dormir, bem viver  altenburg.oficial        257544.0       3414.0   
1         Altenburg Haus      altenburghaus         25719.0        626.0   
2             Artelassê          artelasse        283629.0       3897.0   

                  last_update  \
0  2024-10-03T14:05:50.453915   
1  2024-10-03T14:05:50.875200   
2  2024-10-03T14:05:51.360183   

                                           biography  \
0  Moda Casa com elegância, alta qualidade, tecno...   
1  Desde 1922, trazendo elegância e sofisticação ...   
2  Enxovais premium de cama e banho.\nPara quem v...   

                                         profile_pic  
0  https://scontent-mad1-1.cdninstagram.com/v/t51...  
1  https://scontent-ams4-1.cdninstagram.com/v/t51...  
2  https://scontent-mad2-1.cdninstagram.com/v/t51...  
IEmailProperties(subject='[TESTE] - Seguidores e postagens', body='\n            Report s

True

## Polimentos finais no DataManager

Objetivos:
- Obter usernames unicos dentro da base
- Ordenar a lista por data de atualização
- Obter data da ultima atualização

In [2]:
from innovation_api.api.services.instagram_monitor_service import InstagramMonitorService

service = InstagramMonitorService()
df_user_info = service.data_manager.load_current_account_history_dataset()
df_user_info

,name,username,follower_count,total_media,last_update,biography,profile_pic
16,Zelo,zeloloja,243012.0,2974.0,2024-10-03T15:57:26.128600,"Perfil oficial da Zelo, a mais completa rede e...",https://scontent-ams2-1.cdninstagram.com/v/t51...
15,Trousseau Oficial,trousseauoficial,256493.0,3837.0,2024-10-03T15:57:25.693080,"Desde 1991 no universo de lifestyle, com produ...",https://scontent-ams4-1.cdninstagram.com/v/t51...
14,Trussardi,trussardioficial,294307.0,635.0,2024-10-03T15:57:25.104510,Dal 1898.\nEstado da arte na rotina\nConheças ...,https://scontent-atl3-1.cdninstagram.com/v/t51...
13,Santista,santistadecora,1405760.0,4416.0,2024-10-03T15:57:24.675343,"Seu lar, nosso lugar 💙",https://scontent-mad2-1.cdninstagram.com/v/t51...
12,mmartan,mmartanoficial,1172318.0,4620.0,2024-10-03T15:57:24.216808,"Desde 1980 criando histórias com a sua casa, s...",https://scontent-ams4-1.cdninstagram.com/v/t51...
11,Karsten Oficial,karstenoficial,737238.0,4229.0,2024-10-03T15:57:23.585997,Casa renovada casa com você \nHá 141 anos traz...,https://scontent-mad2-1.cdninstagram.com/v/t51...
10,Hoomy,hoomybr,60416.0,482.0,2024-10-03T15:57:18.130775,O único Lençol do Brasil feito para você dormi...,https://scontent-lhr8-2.cdninstagram.com/v/t51...
9,Casa Sonno,casa.sonno,8877.0,201.0,2024-10-03T15:57:12.644473,Roupas de cama para sonhadores ✸\nFeitas no Br...,https://scontent-ams2-1.cdninstagram.com/v/t51...
8,Casa Riachuelo,casariachuelo,1110618.0,2212.0,2024-10-03T15:57:07.154863,dicas e inspirações para você viver a sua casa...,https://scontent-mad1-1.cdninstagram.com/v/t51...
7,Casa Almeida,casaalmeidaoficial,103756.0,1544.0,2024-10-03T15:57:01.732045,Uma experiência inigualável de sensações e est...,https://scontent-atl3-1.cdninstagram.com/v/t51...


In [3]:
service.update_accounts_info(usernames=["altenburg.oficial",
        "altenburghaus",
        "artelasse",
        "artex",
        "buddemeyeroficial",
        "casaalmeidaoficial",
        "casariachuelo",
        "casa.sonno",
        "hoomybr",
        "karstenoficial",
        "mmartanoficial",
        "santistadecora",
        "trussardioficial",
        "trousseauoficial",
        "zeloloja"])

                     name            username  follower_count  total_media  \
0   Bem dormir, bem viver   altenburg.oficial        257544.0       3414.0   
1                    Zelo            zeloloja        243012.0       2974.0   
2   Bem dormir, bem viver   altenburg.oficial        257544.0       3414.0   
3          Altenburg Haus       altenburghaus         25719.0        626.0   
4              Artelassê           artelasse        283629.0       3897.0   
5                   ARTEX               artex       1029791.0       4083.0   
6              Buddemeyer   buddemeyeroficial        286057.0       1941.0   
7            Casa Almeida  casaalmeidaoficial        103756.0       1544.0   
8          Casa Riachuelo       casariachuelo       1110618.0       2212.0   
9              Casa Sonno          casa.sonno          8877.0        201.0   
10                  Hoomy             hoomybr         60416.0        482.0   
11        Karsten Oficial      karstenoficial        737238.0   

True

In [5]:
df_user_info = service.data_manager.load_current_account_history_dataset()
df_user_info

,name,username,follower_count,total_media,last_update,biography,profile_pic
0,"Bem dormir, bem viver",altenburg.oficial,257544.0,3414.0,2024-10-03T15:25:45.525031,"Moda Casa com elegância, alta qualidade, tecno...",https://scontent-mad1-1.cdninstagram.com/v/t51...
1,Zelo,zeloloja,243012.0,2974.0,2024-10-03T15:25:46.980914,"Perfil oficial da Zelo, a mais completa rede e...",https://scontent-ams2-1.cdninstagram.com/v/t51...
2,"Bem dormir, bem viver",altenburg.oficial,257544.0,3414.0,2024-10-03T15:56:34.393041,"Moda Casa com elegância, alta qualidade, tecno...",https://scontent-mad1-1.cdninstagram.com/v/t51...
3,Altenburg Haus,altenburghaus,25719.0,626.0,2024-10-03T15:56:39.875414,"Desde 1922, trazendo elegância e sofisticação ...",https://scontent-ams4-1.cdninstagram.com/v/t51...
4,Artelassê,artelasse,283629.0,3897.0,2024-10-03T15:56:45.347409,Enxovais premium de cama e banho.\nPara quem v...,https://scontent-mad2-1.cdninstagram.com/v/t51...
5,ARTEX,artex,1029791.0,4083.0,2024-10-03T15:56:50.805291,Viva o conforto de ser você! ❤️\n• Use o cupom...,https://scontent-mad1-1.cdninstagram.com/v/t51...
6,Buddemeyer,buddemeyeroficial,286057.0,1941.0,2024-10-03T15:56:56.249062,Para os melhores momentos em casa!,https://scontent-sin6-4.cdninstagram.com/v/t51...
7,Casa Almeida,casaalmeidaoficial,103756.0,1544.0,2024-10-03T15:57:01.732045,Uma experiência inigualável de sensações e est...,https://scontent-atl3-1.cdninstagram.com/v/t51...
8,Casa Riachuelo,casariachuelo,1110618.0,2212.0,2024-10-03T15:57:07.154863,dicas e inspirações para você viver a sua casa...,https://scontent-mad1-1.cdninstagram.com/v/t51...
9,Casa Sonno,casa.sonno,8877.0,201.0,2024-10-03T15:57:12.644473,Roupas de cama para sonhadores ✸\nFeitas no Br...,https://scontent-ams2-1.cdninstagram.com/v/t51...


In [16]:
import pandas as pd
from typing import List
import datetime

def get_usernames(dataset: pd.DataFrame) -> List[str]:
    usernames = dataset['username'].unique()
    
    return usernames.tolist()

def get_last_update(dataset: pd.DataFrame) -> str:
    max_last_update = dataset['last_update'].max()
    last_update = datetime.datetime.fromisoformat(max_last_update)
    return last_update.strftime("%d-%m-%Y")


res = get_last_update(df_user_info)
res

'03-10-2024'

In [1]:
from innovation_api.api.services.instagram_monitor_service import InstagramMonitorService

service = InstagramMonitorService()

body = service.send_report_xlsx(recipients=['joao.garcia@ammovarejo.com.br'])
print(body)



            Report sobre número de seguidres e número total de postagens.
            
            Ultima atualização: 03-10-2024
            
            Contas observadas: 
            a,l,t,e,n,b,u,r,g,.,o,f,i,c,i,a,l,,, ,a,l,t,e,n,b,u,r,g,h,a,u,s,,, ,a,r,t,e,l,a,s,s,e,,, ,a,r,t,e,x,,, ,b,u,d,d,e,m,e,y,e,r,o,f,i,c,i,a,l,,, ,c,a,s,a,.,s,o,n,n,o,,, ,c,a,s,a,a,l,m,e,i,d,a,o,f,i,c,i,a,l,,, ,c,a,s,a,r,i,a,c,h,u,e,l,o,,, ,h,o,o,m,y,b,r,,, ,k,a,r,s,t,e,n,o,f,i,c,i,a,l,,, ,m,m,a,r,t,a,n,o,f,i,c,i,a,l,,, ,s,a,n,t,i,s,t,a,d,e,c,o,r,a,,, ,t,r,o,u,s,s,e,a,u,o,f,i,c,i,a,l,,, ,t,r,u,s,s,a,r,d,i,o,f,i,c,i,a,l,,, ,z,e,l,o,l,o,j,a
        


In [1]:
from innovation_api.api.services.instagram_monitor_service import InstagramMonitorService

service = InstagramMonitorService()
dataframe = service.data_manager.load_current_account_history_dataset()

In [2]:
from commom.database.data_handler import DataHandler

data_write = DataHandler()
data_write.write_to_bigquery(dataset_id='innovation_dataset', table_id='instagram_accounts_history', dataframe=dataframe)


[INNOVATION-LOG] [to_bigquery] [Line (41)] [INFO]: Connection to BigQuery successful.
[INNOVATION-LOG] [create_table_if_not_exists] [Line (14)] [INFO]: The table does not exist. Creating the table...
[INNOVATION-LOG] [create_table_if_not_exists] [Line (17)] [INFO]: Table created successfully.
[INNOVATION-LOG] [to_bigquery] [Line (51)] [INFO]: write done


,name,username,follower_count,total_media,last_update,biography,profile_pic
16,Zelo,zeloloja,243012.0,2974.0,2024-10-03T15:57:26.128600,"Perfil oficial da Zelo, a mais completa rede e...",https://scontent-ams2-1.cdninstagram.com/v/t51...
15,Trousseau Oficial,trousseauoficial,256493.0,3837.0,2024-10-03T15:57:25.693080,"Desde 1991 no universo de lifestyle, com produ...",https://scontent-ams4-1.cdninstagram.com/v/t51...
14,Trussardi,trussardioficial,294307.0,635.0,2024-10-03T15:57:25.104510,Dal 1898.\nEstado da arte na rotina\nConheças ...,https://scontent-atl3-1.cdninstagram.com/v/t51...
13,Santista,santistadecora,1405760.0,4416.0,2024-10-03T15:57:24.675343,"Seu lar, nosso lugar 💙",https://scontent-mad2-1.cdninstagram.com/v/t51...
12,mmartan,mmartanoficial,1172318.0,4620.0,2024-10-03T15:57:24.216808,"Desde 1980 criando histórias com a sua casa, s...",https://scontent-ams4-1.cdninstagram.com/v/t51...
11,Karsten Oficial,karstenoficial,737238.0,4229.0,2024-10-03T15:57:23.585997,Casa renovada casa com você \nHá 141 anos traz...,https://scontent-mad2-1.cdninstagram.com/v/t51...
10,Hoomy,hoomybr,60416.0,482.0,2024-10-03T15:57:18.130775,O único Lençol do Brasil feito para você dormi...,https://scontent-lhr8-2.cdninstagram.com/v/t51...
9,Casa Sonno,casa.sonno,8877.0,201.0,2024-10-03T15:57:12.644473,Roupas de cama para sonhadores ✸\nFeitas no Br...,https://scontent-ams2-1.cdninstagram.com/v/t51...
8,Casa Riachuelo,casariachuelo,1110618.0,2212.0,2024-10-03T15:57:07.154863,dicas e inspirações para você viver a sua casa...,https://scontent-mad1-1.cdninstagram.com/v/t51...
7,Casa Almeida,casaalmeidaoficial,103756.0,1544.0,2024-10-03T15:57:01.732045,Uma experiência inigualável de sensações e est...,https://scontent-atl3-1.cdninstagram.com/v/t51...


In [5]:
from google.cloud import bigquery
from commom.config import config


client = bigquery.Client(project=config.credentials.project_id, credentials=config.credentials)
client

In [6]:
table_ref = client.dataset('teste_do_joao').table('table_teste')

In [10]:
print(table_ref.dataset_id)
print(table_ref.path)

teste_do_joao
/projects/projetoomni/datasets/teste_do_joao/tables/table_teste


In [12]:
try:
    table = client.get_table(table_ref)
    table
except Exception as e:
    print(e)

403 GET https://bigquery.googleapis.com/bigquery/v2/projects/projetoomni/datasets/teste_do_joao/tables/table_teste?prettyPrint=false: Access Denied: Table projetoomni:teste_do_joao.table_teste: Permission bigquery.tables.get denied on table projetoomni:teste_do_joao.table_teste (or it may not exist).


In [14]:
table = bigquery.Table(table_ref)
table = client.create_table(table)
table

Table(TableReference(DatasetReference('projetoomni', 'teste_do_joao'), 'table_teste'))

In [3]:
dataframe = service.data_manager.load_current_account_history_dataset()

In [15]:
import pandas as pd
dataframe['last_update'] = pd.to_datetime(dataframe['last_update']).dt.strftime('%Y-%m-%d')
mask = dataframe.duplicated(subset=['username', 'last_update'])
dataframe[~mask]

,name,username,follower_count,total_media,last_update,biography,profile_pic
16,Zelo,zeloloja,243012.0,2974.0,2024-10-03,"Perfil oficial da Zelo, a mais completa rede e...",https://scontent-ams2-1.cdninstagram.com/v/t51...
15,Trousseau Oficial,trousseauoficial,256493.0,3837.0,2024-10-03,"Desde 1991 no universo de lifestyle, com produ...",https://scontent-ams4-1.cdninstagram.com/v/t51...
14,Trussardi,trussardioficial,294307.0,635.0,2024-10-03,Dal 1898.\nEstado da arte na rotina\nConheças ...,https://scontent-atl3-1.cdninstagram.com/v/t51...
13,Santista,santistadecora,1405760.0,4416.0,2024-10-03,"Seu lar, nosso lugar 💙",https://scontent-mad2-1.cdninstagram.com/v/t51...
12,mmartan,mmartanoficial,1172318.0,4620.0,2024-10-03,"Desde 1980 criando histórias com a sua casa, s...",https://scontent-ams4-1.cdninstagram.com/v/t51...
11,Karsten Oficial,karstenoficial,737238.0,4229.0,2024-10-03,Casa renovada casa com você \nHá 141 anos traz...,https://scontent-mad2-1.cdninstagram.com/v/t51...
10,Hoomy,hoomybr,60416.0,482.0,2024-10-03,O único Lençol do Brasil feito para você dormi...,https://scontent-lhr8-2.cdninstagram.com/v/t51...
9,Casa Sonno,casa.sonno,8877.0,201.0,2024-10-03,Roupas de cama para sonhadores ✸\nFeitas no Br...,https://scontent-ams2-1.cdninstagram.com/v/t51...
8,Casa Riachuelo,casariachuelo,1110618.0,2212.0,2024-10-03,dicas e inspirações para você viver a sua casa...,https://scontent-mad1-1.cdninstagram.com/v/t51...
7,Casa Almeida,casaalmeidaoficial,103756.0,1544.0,2024-10-03,Uma experiência inigualável de sensações e est...,https://scontent-atl3-1.cdninstagram.com/v/t51...


In [14]:
dataframe['last_update'] = pd.to_datetime(dataframe['last_update']).dt.strftime('%Y-%m-%d')
a = dataframe.drop_duplicates(subset=['username', 'last_update'])
a

,name,username,follower_count,total_media,last_update,biography,profile_pic
16,Zelo,zeloloja,243012.0,2974.0,2024-10-03,"Perfil oficial da Zelo, a mais completa rede e...",https://scontent-ams2-1.cdninstagram.com/v/t51...
15,Trousseau Oficial,trousseauoficial,256493.0,3837.0,2024-10-03,"Desde 1991 no universo de lifestyle, com produ...",https://scontent-ams4-1.cdninstagram.com/v/t51...
14,Trussardi,trussardioficial,294307.0,635.0,2024-10-03,Dal 1898.\nEstado da arte na rotina\nConheças ...,https://scontent-atl3-1.cdninstagram.com/v/t51...
13,Santista,santistadecora,1405760.0,4416.0,2024-10-03,"Seu lar, nosso lugar 💙",https://scontent-mad2-1.cdninstagram.com/v/t51...
12,mmartan,mmartanoficial,1172318.0,4620.0,2024-10-03,"Desde 1980 criando histórias com a sua casa, s...",https://scontent-ams4-1.cdninstagram.com/v/t51...
11,Karsten Oficial,karstenoficial,737238.0,4229.0,2024-10-03,Casa renovada casa com você \nHá 141 anos traz...,https://scontent-mad2-1.cdninstagram.com/v/t51...
10,Hoomy,hoomybr,60416.0,482.0,2024-10-03,O único Lençol do Brasil feito para você dormi...,https://scontent-lhr8-2.cdninstagram.com/v/t51...
9,Casa Sonno,casa.sonno,8877.0,201.0,2024-10-03,Roupas de cama para sonhadores ✸\nFeitas no Br...,https://scontent-ams2-1.cdninstagram.com/v/t51...
8,Casa Riachuelo,casariachuelo,1110618.0,2212.0,2024-10-03,dicas e inspirações para você viver a sua casa...,https://scontent-mad1-1.cdninstagram.com/v/t51...
7,Casa Almeida,casaalmeidaoficial,103756.0,1544.0,2024-10-03,Uma experiência inigualável de sensações e est...,https://scontent-atl3-1.cdninstagram.com/v/t51...


In [20]:
import datetime
datetime.datetime.now().date()

datetime.date(2024, 10, 4)

In [21]:
mask_zelo = dataframe['username'] == 'zeloloja'
mask_al = dataframe['username'] == 'altenburghaus'
df_zelo = dataframe[mask_zelo]
df_alt = dataframe[mask_al]


In [29]:
df_zelo['teste'] = 1

/var/folders/4t/13gtr7_95k58jpj2y37sh5nr0000gr/T/ipykernel_15841/2244140845.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_zelo['teste'] = 1


In [30]:
df_zelo

,name,username,follower_count,total_media,last_update,biography,profile_pic,teste
16,Zelo,zeloloja,243012.0,2974.0,2024-10-03,"Perfil oficial da Zelo, a mais completa rede e...",https://scontent-ams2-1.cdninstagram.com/v/t51...,1
1,Zelo,zeloloja,243012.0,2974.0,2024-10-03,"Perfil oficial da Zelo, a mais completa rede e...",https://scontent-ams2-1.cdninstagram.com/v/t51...,1


In [31]:
df_list = [
    df_zelo,
    df_alt
]


In [32]:
pd.concat(df_list)

,name,username,follower_count,total_media,last_update,biography,profile_pic,teste
16,Zelo,zeloloja,243012.0,2974.0,2024-10-03,"Perfil oficial da Zelo, a mais completa rede e...",https://scontent-ams2-1.cdninstagram.com/v/t51...,1.0
1,Zelo,zeloloja,243012.0,2974.0,2024-10-03,"Perfil oficial da Zelo, a mais completa rede e...",https://scontent-ams2-1.cdninstagram.com/v/t51...,1.0
3,Altenburg Haus,altenburghaus,25719.0,626.0,2024-10-03,"Desde 1922, trazendo elegância e sofisticação ...",https://scontent-ams4-1.cdninstagram.com/v/t51...,NaN


In [25]:
qualquer = [
    1,2,3,4,5,6,7,8,9,10
]

In [27]:
nova_lista = [n+1 for n in qualquer if n > 5]
nova_lista

[7, 8, 9, 10, 11]

## Testando novos métodos e refatoração

- [Y] Load pickle local
- [Y] Fetch data and update pickle local
- [Y] Upload dataframe to BQ
- [Y] Load picle from BQ
- [Y] Fetch data and update BQ
- [Y] Send BQ data to email

In [1]:
from commom.instagram_data.instagram_data_manager import InstagramDataManager
from innovation_api.api.services.instagram_monitor_service import InstagramMonitorService

manager = InstagramDataManager()
service = InstagramMonitorService()


In [6]:
import pandas as pd
import datetime
from commom.instagram_data.instagram_data_manager import InstagramDataManager
from innovation_api.api.services.instagram_monitor_service import InstagramMonitorService

manager = InstagramDataManager()
service = InstagramMonitorService()


df_novo = manager.load_current_account_history_dataset(source='local', file_path='accounts_info.pkl')

In [7]:
df_novo['last_update'] = pd.to_datetime(df_novo['last_update']).dt.strftime('%Y-%m-%d')
df_novo

,name,username,follower_count,total_media,last_update,biography,profile_pic
16,Zelo,zeloloja,243012.0,2974.0,2024-10-03,"Perfil oficial da Zelo, a mais completa rede e...",https://scontent-ams2-1.cdninstagram.com/v/t51...
15,Trousseau Oficial,trousseauoficial,256493.0,3837.0,2024-10-03,"Desde 1991 no universo de lifestyle, com produ...",https://scontent-ams4-1.cdninstagram.com/v/t51...
14,Trussardi,trussardioficial,294307.0,635.0,2024-10-03,Dal 1898.\nEstado da arte na rotina\nConheças ...,https://scontent-atl3-1.cdninstagram.com/v/t51...
13,Santista,santistadecora,1405760.0,4416.0,2024-10-03,"Seu lar, nosso lugar 💙",https://scontent-mad2-1.cdninstagram.com/v/t51...
12,mmartan,mmartanoficial,1172318.0,4620.0,2024-10-03,"Desde 1980 criando histórias com a sua casa, s...",https://scontent-ams4-1.cdninstagram.com/v/t51...
11,Karsten Oficial,karstenoficial,737238.0,4229.0,2024-10-03,Casa renovada casa com você \nHá 141 anos traz...,https://scontent-mad2-1.cdninstagram.com/v/t51...
10,Hoomy,hoomybr,60416.0,482.0,2024-10-03,O único Lençol do Brasil feito para você dormi...,https://scontent-lhr8-2.cdninstagram.com/v/t51...
9,Casa Sonno,casa.sonno,8877.0,201.0,2024-10-03,Roupas de cama para sonhadores ✸\nFeitas no Br...,https://scontent-ams2-1.cdninstagram.com/v/t51...
8,Casa Riachuelo,casariachuelo,1110618.0,2212.0,2024-10-03,dicas e inspirações para você viver a sua casa...,https://scontent-mad1-1.cdninstagram.com/v/t51...
7,Casa Almeida,casaalmeidaoficial,103756.0,1544.0,2024-10-03,Uma experiência inigualável de sensações e est...,https://scontent-atl3-1.cdninstagram.com/v/t51...


In [3]:
l = [
        "altenburg.oficial",
        "altenburghaus",
        "artelasse",
        "artex",
        "buddemeyeroficial",
        "casaalmeidaoficial",
        "casariachuelo",
        "casa.sonno",
        "hoomybr",
        "karstenoficial",
        "mmartanoficial",
        "santistadecora",
        "trussardioficial",
        "trousseauoficial",
        "zeloloja"
    ]
users_info = manager.fetch_accounts_infos(instagram_account_list=l)

[INNOVATION-LOG] [fetch_user_info] [Line (21)] [INFO]: Fetching user info - init
[INNOVATION-LOG] [fetch_user_info] [Line (24)] [INFO]: Service: instagram_scrapper_api
[INNOVATION-LOG] [fetch_user_info] [Line (40)] [INFO]: Fetching user info - end


In [8]:
from commom.data_classes.instagram_data_class import  InstagramAccountInfo
import datetime
u = [InstagramAccountInfo(name='Bem dormir, bem viver', username='altenburg.oficial', follower_count=257642, total_media=3416, last_update=datetime.date(2024, 10, 4), biography='Moda Casa com elegância, alta qualidade, tecnologias e práticas cada vez mais sustentáveis para o seu bem-estar.', profile_pic='https://scontent-cdg4-2.cdninstagram.com/v/t51.2885-19/405204100_311861495005263_4564322676966450610_n.jpg?stp=dst-jpg_s640x640&_nc_ht=scontent-cdg4-2.cdninstagram.com&_nc_cat=103&_nc_ohc=LtGaU_7eWFUQ7kNvgHCEbWy&_nc_gid=98ae0afb2b894f4faa5401f90566077f&edm=AO4kU9EBAAAA&ccb=7-5&oh=00_AYC9ITL-jbFFt87hWNRkS8kXDQ--dU0ikmadQPH2Csu98A&oe=67062AD6&_nc_sid=164c1d'),
 InstagramAccountInfo(name='Altenburg Haus', username='altenburghaus', follower_count=25729, total_media=626, last_update=datetime.date(2024, 10, 4), biography='Desde 1922, trazendo elegância e sofisticação atemporais para sua casa.', profile_pic='https://scontent-fra3-1.cdninstagram.com/v/t51.2885-19/262576167_1531601293888325_7082004465475942676_n.jpg?stp=dst-jpg_s640x640&_nc_ht=scontent-fra3-1.cdninstagram.com&_nc_cat=105&_nc_ohc=liV2pd_wGfsQ7kNvgGbAbVT&_nc_gid=aa08ced7dc2d46f3a29dcdd242ae4529&edm=AO4kU9EBAAAA&ccb=7-5&oh=00_AYCV0xPYJlTU6oBMXD-dLH29pAUozdXOlA953rODeO8PjQ&oe=6706257C&_nc_sid=164c1d'),
 InstagramAccountInfo(name='Artelassê', username='artelasse', follower_count=283655, total_media=3898, last_update=datetime.date(2024, 10, 4), biography='Enxovais premium de cama e banho.\nPara quem valoriza o descanso e o autocuidado.\n40 anos cuidando do seu conforto e\xa0sono\xa0perfeito!', profile_pic='https://scontent-mad2-1.cdninstagram.com/v/t51.2885-19/445750899_1621929715222978_3588756742387398129_n.jpg?stp=dst-jpg_s320x320&_nc_ht=scontent-mad2-1.cdninstagram.com&_nc_cat=1&_nc_ohc=0rOWDV62sNkQ7kNvgEdC72p&_nc_gid=6e0631e2e435407b9af65504d8651fb5&edm=AO4kU9EBAAAA&ccb=7-5&oh=00_AYBsuYr47u9wU_doZici3vVB8rYx6fQ_cDDNzDpVMuD9eA&oe=67060D93&_nc_sid=164c1d'),
 InstagramAccountInfo(name='ARTEX', username='artex', follower_count=1029798, total_media=4085, last_update=datetime.date(2024, 10, 4), biography='Viva o conforto de ser você! ❤️\n• Use o cupom AXBEMVINDO10', profile_pic='https://scontent-ams4-1.cdninstagram.com/v/t51.2885-19/455108042_860014102275169_1657998699154895928_n.jpg?stp=dst-jpg_s640x640&_nc_ht=scontent-ams4-1.cdninstagram.com&_nc_cat=101&_nc_ohc=eAECUEqMujgQ7kNvgGHXB5w&_nc_gid=efc0e714bb244987aad65ab816d8f173&edm=AO4kU9EBAAAA&ccb=7-5&oh=00_AYBzltVLE-I5n1mJcvq1doGbK9LC9wX18IhCFEIEidGMPA&oe=6705F681&_nc_sid=164c1d'),
 InstagramAccountInfo(name='Buddemeyer', username='buddemeyeroficial', follower_count=286068, total_media=1941, last_update=datetime.date(2024, 10, 4), biography='Para os melhores momentos em casa!', profile_pic='https://scontent.cdninstagram.com/v/t51.2885-19/289682394_2265494573613032_8652412004800493322_n.jpg?stp=dst-jpg_s640x640&_nc_ht=scontent.cdninstagram.com&_nc_cat=108&_nc_ohc=cayeZSqiZgMQ7kNvgEe23wP&_nc_gid=736f156a4cf3445ba2e8d05aefaaf338&edm=AO4kU9EBAAAA&ccb=7-5&oh=00_AYC33n11o6tOsnYK4cz4trZrju30b-nBLRZvhdARrjJeig&oe=67062570&_nc_sid=164c1d'),
 InstagramAccountInfo(name='Casa Almeida', username='casaalmeidaoficial', follower_count=103762, total_media=1545, last_update=datetime.date(2024, 10, 4), biography='Uma experiência inigualável de sensações e estilo!', profile_pic='https://scontent-ams4-1.cdninstagram.com/v/t51.2885-19/34703221_1956236587754446_834498736724901888_n.jpg?stp=dst-jpg_s640x640&_nc_ht=scontent-ams4-1.cdninstagram.com&_nc_cat=110&_nc_ohc=3eJL4ySO3pYQ7kNvgFyWGz9&_nc_gid=3e047a455ab742a5bfdf06135b78f163&edm=AO4kU9EBAAAA&ccb=7-5&oh=00_AYB6bsNrdHa1pynxbhusgkcogy6a6owmVxKInLgME5H-Mg&oe=67061F19&_nc_sid=164c1d'),
 InstagramAccountInfo(name='Casa Riachuelo', username='casariachuelo', follower_count=1110654, total_media=2213, last_update=datetime.date(2024, 10, 4), biography='dicas e inspirações para você viver a sua casa de um jeito especial. 🏠 ✨', profile_pic='https://scontent-atl3-2.cdninstagram.com/v/t51.2885-19/92350347_237345247643174_3460258913331445760_n.jpg?stp=dst-jpg_s320x320&_nc_ht=scontent-atl3-2.cdninstagram.com&_nc_cat=102&_nc_ohc=EnfxT9QvKU4Q7kNvgHcTclw&_nc_gid=e16d5152b123488c9078f2a6c7fa17b1&edm=AO4kU9EBAAAA&ccb=7-5&oh=00_AYAsAAqDZvLhRfjQaQc1Y6uG5v2YFYByLmqoUjoikZiG8w&oe=67062C1F&_nc_sid=164c1d'),
 InstagramAccountInfo(name='Casa Sonno', username='casa.sonno', follower_count=8894, total_media=201, last_update=datetime.date(2024, 10, 4), biography='Roupas de cama para sonhadores ✸\nFeitas no Brasil\nShop now ↓', profile_pic='https://scontent-lhr6-2.cdninstagram.com/v/t51.2885-19/341393986_1553390998483044_2412731059355813972_n.jpg?stp=dst-jpg_s640x640&_nc_ht=scontent-lhr6-2.cdninstagram.com&_nc_cat=100&_nc_ohc=7B32t8BN9C8Q7kNvgGlORU3&_nc_gid=46ed43f7eb9849a6ba5e5679e926c7da&edm=AO4kU9EBAAAA&ccb=7-5&oh=00_AYA8Uu_RuneRABPk2--LIrImCVU691gnInsYEApCC7y-wQ&oe=67062051&_nc_sid=164c1d'),
 InstagramAccountInfo(name='Hoomy', username='hoomybr', follower_count=60576, total_media=483, last_update=datetime.date(2024, 10, 4), biography='O único Lençol do Brasil feito para você dormir melhor e a Toalha turca mais fofa do país!\n—\nClique no link para comprar 👇', profile_pic='https://scontent-ams2-1.cdninstagram.com/v/t51.2885-19/457778998_1576420316646210_3765466817309600786_n.jpg?stp=dst-jpg_s640x640&_nc_ht=scontent-ams2-1.cdninstagram.com&_nc_cat=106&_nc_ohc=2RDayQHoY30Q7kNvgE7iAc5&_nc_gid=480ccf47ad884726b271502422d72cd5&edm=AO4kU9EBAAAA&ccb=7-5&oh=00_AYCqzgvHRfh7LwDlhjPne5lWsxVPB8GIH8SfGgVYdHOXZQ&oe=67062AF5&_nc_sid=164c1d'),
 InstagramAccountInfo(name='Karsten Oficial', username='karstenoficial', follower_count=737290, total_media=4231, last_update=datetime.date(2024, 10, 4), biography='Casa renovada casa com você \nHá 141 anos trazendo um toque de conforto em cada detalhe. Sinta-se em casa 💙', profile_pic='https://scontent-mad2-1.cdninstagram.com/v/t51.2885-19/345913952_1014306869732536_903903603524720529_n.jpg?stp=dst-jpg_s640x640&_nc_ht=scontent-mad2-1.cdninstagram.com&_nc_cat=104&_nc_ohc=yQ_yeHCLmPcQ7kNvgH7i9k-&_nc_gid=8f39cf08b7574adf8116c6466a8fc223&edm=AO4kU9EBAAAA&ccb=7-5&oh=00_AYDVAuF99F3IyB9FXoVm2Hrmd1vP8WL0SbVkDfc_AWKwlg&oe=67060259&_nc_sid=164c1d'),
 InstagramAccountInfo(name='mmartan', username='mmartanoficial', follower_count=1172367, total_media=4621, last_update=datetime.date(2024, 10, 4), biography='Desde 1980 criando histórias com a sua casa, seja no toque macio dos lençóis, banho aconchegante ou aromas inconfundíveis.', profile_pic='https://scontent-ams4-1.cdninstagram.com/v/t51.2885-19/404293436_766960741933447_2962518092099297043_n.jpg?stp=dst-jpg_s640x640&_nc_ht=scontent-ams4-1.cdninstagram.com&_nc_cat=107&_nc_ohc=1NsIvSyXtEQQ7kNvgHbsyBR&_nc_gid=02aba14097534b83bf70558b77b4dc8a&edm=AO4kU9EBAAAA&ccb=7-5&oh=00_AYDDaSVMSTC7ybbTYiZa_T4ppIgJDfck8FKPwOTIfg67iw&oe=670610B1&_nc_sid=164c1d'),
 InstagramAccountInfo(name='Santista', username='santistadecora', follower_count=1405642, total_media=4417, last_update=datetime.date(2024, 10, 4), biography='Seu lar, nosso lugar 💙', profile_pic='https://scontent-fra5-2.cdninstagram.com/v/t51.2885-19/404885874_328526089932705_8322732610566886133_n.jpg?stp=dst-jpg_s640x640&_nc_ht=scontent-fra5-2.cdninstagram.com&_nc_cat=109&_nc_ohc=axn4TbzUzcQQ7kNvgEZIlnk&_nc_gid=81a8b60d3c8e4dcbaaaa8a2139364cc5&edm=AO4kU9EBAAAA&ccb=7-5&oh=00_AYDv30vqfrQChhQ75zcmPL8sRO595O4cjlAvdD0hl4A9kA&oe=67060898&_nc_sid=164c1d'),
 InstagramAccountInfo(name='Trussardi', username='trussardioficial', follower_count=294397, total_media=637, last_update=datetime.date(2024, 10, 4), biography='Dal 1898.\nEstado da arte na rotina\nConheças nossas lojas: São Paulo, Curitiba, Campo Largo, Brasília e Porto Belo.', profile_pic='https://scontent-fra5-1.cdninstagram.com/v/t51.2885-19/453568447_1161029788338532_402977306230369935_n.jpg?stp=dst-jpg_s640x640&_nc_ht=scontent-fra5-1.cdninstagram.com&_nc_cat=1&_nc_ohc=y2f6z0RZ6cEQ7kNvgF9LYtt&_nc_gid=043dc795457e48b184fd573c7a05a030&edm=AO4kU9EBAAAA&ccb=7-5&oh=00_AYCYuCwsZUc91MU_26lRzEslak-h7KnAwDB0U2HU3LD-qg&oe=67061F3A&_nc_sid=164c1d'),
 InstagramAccountInfo(name='Trousseau Oficial', username='trousseauoficial', follower_count=256535, total_media=3838, last_update=datetime.date(2024, 10, 4), biography='Desde 1991 no universo de lifestyle, com produtos premium de cama, banho, fragrâncias para casa, entre outros.', profile_pic='https://scontent-ams4-1.cdninstagram.com/v/t51.2885-19/271404018_252793856983126_5267776868860026477_n.jpg?stp=dst-jpg_s640x640&_nc_ht=scontent-ams4-1.cdninstagram.com&_nc_cat=110&_nc_ohc=_Jg79N8neNAQ7kNvgG4ULEO&_nc_gid=c71da0a9ccba470d859e671ac12cb33e&edm=AO4kU9EBAAAA&ccb=7-5&oh=00_AYCpssu6djwBzCgoM1YkA21sKvu9zhKrFzF4WV-R0aUxeQ&oe=67061C71&_nc_sid=164c1d'),
 InstagramAccountInfo(name='Zelo', username='zeloloja', follower_count=243041, total_media=2975, last_update=datetime.date(2024, 10, 4), biography='Perfil oficial da Zelo, a mais completa rede especializada em cama, mesa e banho do Brasil.', profile_pic='https://scontent-atl3-1.cdninstagram.com/v/t51.2885-19/460159523_516877664613932_2282200909243022485_n.jpg?stp=dst-jpg_s640x640&_nc_ht=scontent-atl3-1.cdninstagram.com&_nc_cat=108&_nc_ohc=0Qmfnk7H_U0Q7kNvgGkJ7Uo&_nc_gid=984435272c6c425581eae5186e9194a1&edm=AO4kU9EBAAAA&ccb=7-5&oh=00_AYD9bxizUQ9IIF_R5SNf17F4xor6gGDAaUNIi1TaAKyAhw&oe=6705FE6B&_nc_sid=164c1d')]

In [1]:
users_info

NameError: name 'users_info' is not defined

In [9]:
df_novo_updated = manager.update_accounts_history_dataset(df_novo, u)
df_novo_updated

,name,username,follower_count,total_media,last_update,biography,profile_pic
0,Zelo,zeloloja,243012.0,2974.0,2024-10-03,"Perfil oficial da Zelo, a mais completa rede e...",https://scontent-ams2-1.cdninstagram.com/v/t51...
1,Zelo,zeloloja,243012.0,2974.0,2024-10-03,"Perfil oficial da Zelo, a mais completa rede e...",https://scontent-ams2-1.cdninstagram.com/v/t51...
2,Trussardi,trussardioficial,294307.0,635.0,2024-10-03,Dal 1898.\nEstado da arte na rotina\nConheças ...,https://scontent-atl3-1.cdninstagram.com/v/t51...
3,Trousseau Oficial,trousseauoficial,256493.0,3837.0,2024-10-03,"Desde 1991 no universo de lifestyle, com produ...",https://scontent-ams4-1.cdninstagram.com/v/t51...
4,Santista,santistadecora,1405760.0,4416.0,2024-10-03,"Seu lar, nosso lugar 💙",https://scontent-mad2-1.cdninstagram.com/v/t51...
5,mmartan,mmartanoficial,1172318.0,4620.0,2024-10-03,"Desde 1980 criando histórias com a sua casa, s...",https://scontent-ams4-1.cdninstagram.com/v/t51...
6,Karsten Oficial,karstenoficial,737238.0,4229.0,2024-10-03,Casa renovada casa com você \nHá 141 anos traz...,https://scontent-mad2-1.cdninstagram.com/v/t51...
7,Hoomy,hoomybr,60416.0,482.0,2024-10-03,O único Lençol do Brasil feito para você dormi...,https://scontent-lhr8-2.cdninstagram.com/v/t51...
8,Casa Riachuelo,casariachuelo,1110618.0,2212.0,2024-10-03,dicas e inspirações para você viver a sua casa...,https://scontent-mad1-1.cdninstagram.com/v/t51...
9,Casa Almeida,casaalmeidaoficial,103756.0,1544.0,2024-10-03,Uma experiência inigualável de sensações e est...,https://scontent-atl3-1.cdninstagram.com/v/t51...


In [10]:
save = manager.save_current_account_history_dataset(source='local', data=df_novo_updated, path='accounts_info.pkl')


True

In [4]:
import pandas as pd
import datetime
from commom.instagram_data.instagram_data_manager import InstagramDataManager
from innovation_api.api.services.instagram_monitor_service import InstagramMonitorService

manager = InstagramDataManager()
service = InstagramMonitorService()


df_hoje = manager.load_current_account_history_dataset(source='local', file_path='accounts_info.pkl')
df_hoje = df_hoje.drop_duplicates(subset=['username', 'last_update']).reset_index(drop=True)
df_hoje

,name,username,follower_count,total_media,last_update,biography,profile_pic
0,"Bem dormir, bem viver",altenburg.oficial,257642.0,3416.0,2024-10-04,"Moda Casa com elegância, alta qualidade, tecno...",https://scontent-cdg4-2.cdninstagram.com/v/t51...
1,Altenburg Haus,altenburghaus,25729.0,626.0,2024-10-04,"Desde 1922, trazendo elegância e sofisticação ...",https://scontent-fra3-1.cdninstagram.com/v/t51...
2,Artelassê,artelasse,283655.0,3898.0,2024-10-04,Enxovais premium de cama e banho.\nPara quem v...,https://scontent-mad2-1.cdninstagram.com/v/t51...
3,ARTEX,artex,1029798.0,4085.0,2024-10-04,Viva o conforto de ser você! ❤️\n• Use o cupom...,https://scontent-ams4-1.cdninstagram.com/v/t51...
4,Buddemeyer,buddemeyeroficial,286068.0,1941.0,2024-10-04,Para os melhores momentos em casa!,https://scontent.cdninstagram.com/v/t51.2885-1...
5,Casa Sonno,casa.sonno,8894.0,201.0,2024-10-04,Roupas de cama para sonhadores ✸\nFeitas no Br...,https://scontent-lhr6-2.cdninstagram.com/v/t51...
6,Casa Almeida,casaalmeidaoficial,103762.0,1545.0,2024-10-04,Uma experiência inigualável de sensações e est...,https://scontent-ams4-1.cdninstagram.com/v/t51...
7,Casa Riachuelo,casariachuelo,1110654.0,2213.0,2024-10-04,dicas e inspirações para você viver a sua casa...,https://scontent-atl3-2.cdninstagram.com/v/t51...
8,Hoomy,hoomybr,60576.0,483.0,2024-10-04,O único Lençol do Brasil feito para você dormi...,https://scontent-ams2-1.cdninstagram.com/v/t51...
9,Karsten Oficial,karstenoficial,737290.0,4231.0,2024-10-04,Casa renovada casa com você \nHá 141 anos traz...,https://scontent-mad2-1.cdninstagram.com/v/t51...


In [5]:
manager.save_current_account_history_dataset(source='local', data=df_hoje, path='accounts_info.pkl')

True

In [6]:
df_hoje = manager.load_current_account_history_dataset(source='local', file_path='accounts_info.pkl')
df_hoje

,name,username,follower_count,total_media,last_update,biography,profile_pic
0,"Bem dormir, bem viver",altenburg.oficial,257642.0,3416.0,2024-10-04,"Moda Casa com elegância, alta qualidade, tecno...",https://scontent-cdg4-2.cdninstagram.com/v/t51...
8,Hoomy,hoomybr,60576.0,483.0,2024-10-04,O único Lençol do Brasil feito para você dormi...,https://scontent-ams2-1.cdninstagram.com/v/t51...
1,Altenburg Haus,altenburghaus,25729.0,626.0,2024-10-04,"Desde 1922, trazendo elegância e sofisticação ...",https://scontent-fra3-1.cdninstagram.com/v/t51...
14,Zelo,zeloloja,243041.0,2975.0,2024-10-04,"Perfil oficial da Zelo, a mais completa rede e...",https://scontent-atl3-1.cdninstagram.com/v/t51...
13,Trussardi,trussardioficial,294397.0,637.0,2024-10-04,Dal 1898.\nEstado da arte na rotina\nConheças ...,https://scontent-fra5-1.cdninstagram.com/v/t51...
12,Trousseau Oficial,trousseauoficial,256535.0,3838.0,2024-10-04,"Desde 1991 no universo de lifestyle, com produ...",https://scontent-ams4-1.cdninstagram.com/v/t51...
10,mmartan,mmartanoficial,1172367.0,4621.0,2024-10-04,"Desde 1980 criando histórias com a sua casa, s...",https://scontent-ams4-1.cdninstagram.com/v/t51...
9,Karsten Oficial,karstenoficial,737290.0,4231.0,2024-10-04,Casa renovada casa com você \nHá 141 anos traz...,https://scontent-mad2-1.cdninstagram.com/v/t51...
11,Santista,santistadecora,1405642.0,4417.0,2024-10-04,"Seu lar, nosso lugar 💙",https://scontent-fra5-2.cdninstagram.com/v/t51...
7,Casa Riachuelo,casariachuelo,1110654.0,2213.0,2024-10-04,dicas e inspirações para você viver a sua casa...,https://scontent-atl3-2.cdninstagram.com/v/t51...


In [7]:
manager.save_current_account_history_dataset(source='bigquery', dataset_id='innovation_dataset', table_id='instagram_accounts_history', dataframe=df_hoje)

[INNOVATION-LOG] [write_to_bigquery] [Line (29)] [INFO]: Connection to BigQuery successful
[INNOVATION-LOG] [create_table_if_not_exists] [Line (14)] [INFO]: The table does not exist. Creating the table...
[INNOVATION-LOG] [create_table_if_not_exists] [Line (17)] [INFO]: Table created successfully.
[INNOVATION-LOG] [write_to_bigquery] [Line (39)] [INFO]: write done


True

In [1]:
import pandas as pd
import datetime
from commom.instagram_data.instagram_data_manager import InstagramDataManager
from innovation_api.api.services.instagram_monitor_service import InstagramMonitorService

manager = InstagramDataManager()
# service = InstagramMonitorService()


# df_hoje = manager.load_current_account_history_dataset(source='local', file_path='accounts_info.pkl')
tesste = manager._create_empty_accounts_history_dataset()
tesste

,name,username,follower_count,total_media,last_update,biography,profile_pic


In [3]:
from innovation_messenger import IEmailProperties, Messenger
recipients = ['joao.garcia@ammovarejo.com.br']

last_update = manager.get_last_update(df_hoje)
usernames = manager.get_usernames(df_hoje)

file_name = f'Seguidores_e_postagens_atualizado_{last_update}.xlsx'

df_hoje.to_excel(file_name, sheet_name='Seguidores e postagens', index=False)

email_body = f"""
Report sobre número de seguidres e número total de postagens.

Ultima atualização: {last_update}

Contas observadas: 
{sorted(usernames).__str__().replace('[', '').replace(']', '').replace("'", "")}
"""

email_subject = "[TESTE] - Seguidores e postagens - V0.0.1.alpha"
email_properties = IEmailProperties(
    subject=email_subject,
    recipient=recipients,
    file_name=file_name,
    body=email_body
)

Messenger().send_message(
    channel='email',
    email_properties=email_properties
)


True

In [1]:
from commom.data_classes.instagram_data_class import InstagramAccountInfo
from innovation_messenger import IEmailProperties, Messenger
import pandas as pd
import datetime
from commom.instagram_data.instagram_data_manager import InstagramDataManager
from innovation_api.api.services.instagram_monitor_service import InstagramMonitorService
usernames = [
        "altenburg.oficial",
        "altenburghaus",
        "artelasse",
        "artex",
        "buddemeyeroficial",
        "casaalmeidaoficial",
        "casariachuelo",
        "casa.sonno",
        "hoomybr",
        "karstenoficial",
        "mmartanoficial",
        "santistadecora",
        "trussardioficial",
        "trousseauoficial",
        "zeloloja"
    ]
manager = InstagramDataManager()
service = InstagramMonitorService(messenger=Messenger())


response = service.update_accounts_info(usernames=usernames, debug_mode=False)
response

[INNOVATION-LOG] [update_accounts_info] [Line (38)] [INFO]: DEBUG MODE False
[INNOVATION-LOG] [update_accounts_info] [Line (52)] [INFO]: Loading accounts info - init


/Users/administrador/Library/Caches/pypoetry/virtualenvs/innovation-api-kUUtbMIs-py3.10/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


[INNOVATION-LOG] [update_accounts_info] [Line (55)] [INFO]: Updating accounts info - init
[INNOVATION-LOG] [update_accounts_info] [Line (58)] [INFO]: Saving accounts info - init
[INNOVATION-LOG] [write_to_bigquery] [Line (29)] [INFO]: Connection to BigQuery successful


/Users/administrador/Desktop/ammo/innovation-api/src/commom/instagram_data/instagram_data_manager.py:57: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_empty = pd.concat([df_empty, df_account_info]).reset_index(drop=True)


[INNOVATION-LOG] [write_to_bigquery] [Line (39)] [INFO]: write done


{'result': 'Success',
 'message': {'usernames': ['altenburg.oficial',
   'altenburghaus',
   'artelasse',
   'artex',
   'buddemeyeroficial',
   'casaalmeidaoficial',
   'casariachuelo',
   'casa.sonno',
   'hoomybr',
   'karstenoficial',
   'mmartanoficial',
   'santistadecora',
   'trussardioficial',
   'trousseauoficial',
   'zeloloja'],
  'updated_dataset': True}}

In [3]:
from commom.data_classes.instagram_data_class import InstagramAccountInfo
from innovation_messenger import IEmailProperties, Messenger
from commom.database.queries.query_instagram_monitor import QUERY_INSTAGRAM_MONITOR
import pandas as pd
import datetime
from commom.instagram_data.instagram_data_manager import InstagramDataManager
from innovation_api.api.services.instagram_monitor_service import InstagramMonitorService

usernames = [
        "altenburg.oficial",
        "altenburghaus",
        "artelasse",
        "artex",
        "buddemeyeroficial",
        "casaalmeidaoficial",
        "casariachuelo",
        "casa.sonno",
        "hoomybr",
        "karstenoficial",
        "mmartanoficial",
        "santistadecora",
        "trussardioficial",
        "trousseauoficial",
        "zeloloja"
    ]
account_info_list = [InstagramAccountInfo(name='Bem dormir, bem viver', username='altenburg.oficial', follower_count=257642, total_media=3416, last_update=datetime.date(2024, 10, 5), biography='Moda Casa com elegância, alta qualidade, tecnologias e práticas cada vez mais sustentáveis para o seu bem-estar.', profile_pic='https://scontent-cdg4-2.cdninstagram.com/v/t51.2885-19/405204100_311861495005263_4564322676966450610_n.jpg?stp=dst-jpg_s640x640&_nc_ht=scontent-cdg4-2.cdninstagram.com&_nc_cat=103&_nc_ohc=LtGaU_7eWFUQ7kNvgHCEbWy&_nc_gid=98ae0afb2b894f4faa5401f90566077f&edm=AO4kU9EBAAAA&ccb=7-5&oh=00_AYC9ITL-jbFFt87hWNRkS8kXDQ--dU0ikmadQPH2Csu98A&oe=67062AD6&_nc_sid=164c1d'),
 InstagramAccountInfo(name='Altenburg Haus', username='altenburghaus', follower_count=25729, total_media=626, last_update=datetime.date(2024, 10, 5), biography='Desde 1922, trazendo elegância e sofisticação atemporais para sua casa.', profile_pic='https://scontent-fra3-1.cdninstagram.com/v/t51.2885-19/262576167_1531601293888325_7082004465475942676_n.jpg?stp=dst-jpg_s640x640&_nc_ht=scontent-fra3-1.cdninstagram.com&_nc_cat=105&_nc_ohc=liV2pd_wGfsQ7kNvgGbAbVT&_nc_gid=aa08ced7dc2d46f3a29dcdd242ae4529&edm=AO4kU9EBAAAA&ccb=7-5&oh=00_AYCV0xPYJlTU6oBMXD-dLH29pAUozdXOlA953rODeO8PjQ&oe=6706257C&_nc_sid=164c1d'),
 InstagramAccountInfo(name='Artelassê', username='artelasse', follower_count=283655, total_media=3898, last_update=datetime.date(2024, 10, 5), biography='Enxovais premium de cama e banho.\nPara quem valoriza o descanso e o autocuidado.\n40 anos cuidando do seu conforto e\xa0sono\xa0perfeito!', profile_pic='https://scontent-mad2-1.cdninstagram.com/v/t51.2885-19/445750899_1621929715222978_3588756742387398129_n.jpg?stp=dst-jpg_s320x320&_nc_ht=scontent-mad2-1.cdninstagram.com&_nc_cat=1&_nc_ohc=0rOWDV62sNkQ7kNvgEdC72p&_nc_gid=6e0631e2e435407b9af65504d8651fb5&edm=AO4kU9EBAAAA&ccb=7-5&oh=00_AYBsuYr47u9wU_doZici3vVB8rYx6fQ_cDDNzDpVMuD9eA&oe=67060D93&_nc_sid=164c1d'),
 InstagramAccountInfo(name='ARTEX', username='artex', follower_count=1029798, total_media=4085, last_update=datetime.date(2024, 10, 5), biography='Viva o conforto de ser você! ❤️\n• Use o cupom AXBEMVINDO10', profile_pic='https://scontent-ams4-1.cdninstagram.com/v/t51.2885-19/455108042_860014102275169_1657998699154895928_n.jpg?stp=dst-jpg_s640x640&_nc_ht=scontent-ams4-1.cdninstagram.com&_nc_cat=101&_nc_ohc=eAECUEqMujgQ7kNvgGHXB5w&_nc_gid=efc0e714bb244987aad65ab816d8f173&edm=AO4kU9EBAAAA&ccb=7-5&oh=00_AYBzltVLE-I5n1mJcvq1doGbK9LC9wX18IhCFEIEidGMPA&oe=6705F681&_nc_sid=164c1d'),
 InstagramAccountInfo(name='Buddemeyer', username='buddemeyeroficial', follower_count=286068, total_media=1941, last_update=datetime.date(2024, 10, 5), biography='Para os melhores momentos em casa!', profile_pic='https://scontent.cdninstagram.com/v/t51.2885-19/289682394_2265494573613032_8652412004800493322_n.jpg?stp=dst-jpg_s640x640&_nc_ht=scontent.cdninstagram.com&_nc_cat=108&_nc_ohc=cayeZSqiZgMQ7kNvgEe23wP&_nc_gid=736f156a4cf3445ba2e8d05aefaaf338&edm=AO4kU9EBAAAA&ccb=7-5&oh=00_AYC33n11o6tOsnYK4cz4trZrju30b-nBLRZvhdARrjJeig&oe=67062570&_nc_sid=164c1d'),
 InstagramAccountInfo(name='Casa Almeida', username='casaalmeidaoficial', follower_count=103762, total_media=1545, last_update=datetime.date(2024, 10, 5), biography='Uma experiência inigualável de sensações e estilo!', profile_pic='https://scontent-ams4-1.cdninstagram.com/v/t51.2885-19/34703221_1956236587754446_834498736724901888_n.jpg?stp=dst-jpg_s640x640&_nc_ht=scontent-ams4-1.cdninstagram.com&_nc_cat=110&_nc_ohc=3eJL4ySO3pYQ7kNvgFyWGz9&_nc_gid=3e047a455ab742a5bfdf06135b78f163&edm=AO4kU9EBAAAA&ccb=7-5&oh=00_AYB6bsNrdHa1pynxbhusgkcogy6a6owmVxKInLgME5H-Mg&oe=67061F19&_nc_sid=164c1d'),
 InstagramAccountInfo(name='Casa Riachuelo', username='casariachuelo', follower_count=1110654, total_media=2213, last_update=datetime.date(2024, 10, 5), biography='dicas e inspirações para você viver a sua casa de um jeito especial. 🏠 ✨', profile_pic='https://scontent-atl3-2.cdninstagram.com/v/t51.2885-19/92350347_237345247643174_3460258913331445760_n.jpg?stp=dst-jpg_s320x320&_nc_ht=scontent-atl3-2.cdninstagram.com&_nc_cat=102&_nc_ohc=EnfxT9QvKU4Q7kNvgHcTclw&_nc_gid=e16d5152b123488c9078f2a6c7fa17b1&edm=AO4kU9EBAAAA&ccb=7-5&oh=00_AYAsAAqDZvLhRfjQaQc1Y6uG5v2YFYByLmqoUjoikZiG8w&oe=67062C1F&_nc_sid=164c1d'),
 InstagramAccountInfo(name='Casa Sonno', username='casa.sonno', follower_count=8894, total_media=201, last_update=datetime.date(2024, 10, 5), biography='Roupas de cama para sonhadores ✸\nFeitas no Brasil\nShop now ↓', profile_pic='https://scontent-lhr6-2.cdninstagram.com/v/t51.2885-19/341393986_1553390998483044_2412731059355813972_n.jpg?stp=dst-jpg_s640x640&_nc_ht=scontent-lhr6-2.cdninstagram.com&_nc_cat=100&_nc_ohc=7B32t8BN9C8Q7kNvgGlORU3&_nc_gid=46ed43f7eb9849a6ba5e5679e926c7da&edm=AO4kU9EBAAAA&ccb=7-5&oh=00_AYA8Uu_RuneRABPk2--LIrImCVU691gnInsYEApCC7y-wQ&oe=67062051&_nc_sid=164c1d'),
 InstagramAccountInfo(name='Hoomy', username='hoomybr', follower_count=60576, total_media=483, last_update=datetime.date(2024, 10, 5), biography='O único Lençol do Brasil feito para você dormir melhor e a Toalha turca mais fofa do país!\n—\nClique no link para comprar 👇', profile_pic='https://scontent-ams2-1.cdninstagram.com/v/t51.2885-19/457778998_1576420316646210_3765466817309600786_n.jpg?stp=dst-jpg_s640x640&_nc_ht=scontent-ams2-1.cdninstagram.com&_nc_cat=106&_nc_ohc=2RDayQHoY30Q7kNvgE7iAc5&_nc_gid=480ccf47ad884726b271502422d72cd5&edm=AO4kU9EBAAAA&ccb=7-5&oh=00_AYCqzgvHRfh7LwDlhjPne5lWsxVPB8GIH8SfGgVYdHOXZQ&oe=67062AF5&_nc_sid=164c1d'),
 InstagramAccountInfo(name='Karsten Oficial', username='karstenoficial', follower_count=737290, total_media=4231, last_update=datetime.date(2024, 10, 5), biography='Casa renovada casa com você \nHá 141 anos trazendo um toque de conforto em cada detalhe. Sinta-se em casa 💙', profile_pic='https://scontent-mad2-1.cdninstagram.com/v/t51.2885-19/345913952_1014306869732536_903903603524720529_n.jpg?stp=dst-jpg_s640x640&_nc_ht=scontent-mad2-1.cdninstagram.com&_nc_cat=104&_nc_ohc=yQ_yeHCLmPcQ7kNvgH7i9k-&_nc_gid=8f39cf08b7574adf8116c6466a8fc223&edm=AO4kU9EBAAAA&ccb=7-5&oh=00_AYDVAuF99F3IyB9FXoVm2Hrmd1vP8WL0SbVkDfc_AWKwlg&oe=67060259&_nc_sid=164c1d'),
 InstagramAccountInfo(name='mmartan', username='mmartanoficial', follower_count=1172367, total_media=4621, last_update=datetime.date(2024, 10, 5), biography='Desde 1980 criando histórias com a sua casa, seja no toque macio dos lençóis, banho aconchegante ou aromas inconfundíveis.', profile_pic='https://scontent-ams4-1.cdninstagram.com/v/t51.2885-19/404293436_766960741933447_2962518092099297043_n.jpg?stp=dst-jpg_s640x640&_nc_ht=scontent-ams4-1.cdninstagram.com&_nc_cat=107&_nc_ohc=1NsIvSyXtEQQ7kNvgHbsyBR&_nc_gid=02aba14097534b83bf70558b77b4dc8a&edm=AO4kU9EBAAAA&ccb=7-5&oh=00_AYDDaSVMSTC7ybbTYiZa_T4ppIgJDfck8FKPwOTIfg67iw&oe=670610B1&_nc_sid=164c1d'),
 InstagramAccountInfo(name='Santista', username='santistadecora', follower_count=1405642, total_media=4417, last_update=datetime.date(2024, 10, 5), biography='Seu lar, nosso lugar 💙', profile_pic='https://scontent-fra5-2.cdninstagram.com/v/t51.2885-19/404885874_328526089932705_8322732610566886133_n.jpg?stp=dst-jpg_s640x640&_nc_ht=scontent-fra5-2.cdninstagram.com&_nc_cat=109&_nc_ohc=axn4TbzUzcQQ7kNvgEZIlnk&_nc_gid=81a8b60d3c8e4dcbaaaa8a2139364cc5&edm=AO4kU9EBAAAA&ccb=7-5&oh=00_AYDv30vqfrQChhQ75zcmPL8sRO595O4cjlAvdD0hl4A9kA&oe=67060898&_nc_sid=164c1d'),
 InstagramAccountInfo(name='Trussardi', username='trussardioficial', follower_count=294397, total_media=637, last_update=datetime.date(2024, 10, 5), biography='Dal 1898.\nEstado da arte na rotina\nConheças nossas lojas: São Paulo, Curitiba, Campo Largo, Brasília e Porto Belo.', profile_pic='https://scontent-fra5-1.cdninstagram.com/v/t51.2885-19/453568447_1161029788338532_402977306230369935_n.jpg?stp=dst-jpg_s640x640&_nc_ht=scontent-fra5-1.cdninstagram.com&_nc_cat=1&_nc_ohc=y2f6z0RZ6cEQ7kNvgF9LYtt&_nc_gid=043dc795457e48b184fd573c7a05a030&edm=AO4kU9EBAAAA&ccb=7-5&oh=00_AYCYuCwsZUc91MU_26lRzEslak-h7KnAwDB0U2HU3LD-qg&oe=67061F3A&_nc_sid=164c1d'),
 InstagramAccountInfo(name='Trousseau Oficial', username='trousseauoficial', follower_count=256535, total_media=3838, last_update=datetime.date(2024, 10, 5), biography='Desde 1991 no universo de lifestyle, com produtos premium de cama, banho, fragrâncias para casa, entre outros.', profile_pic='https://scontent-ams4-1.cdninstagram.com/v/t51.2885-19/271404018_252793856983126_5267776868860026477_n.jpg?stp=dst-jpg_s640x640&_nc_ht=scontent-ams4-1.cdninstagram.com&_nc_cat=110&_nc_ohc=_Jg79N8neNAQ7kNvgG4ULEO&_nc_gid=c71da0a9ccba470d859e671ac12cb33e&edm=AO4kU9EBAAAA&ccb=7-5&oh=00_AYCpssu6djwBzCgoM1YkA21sKvu9zhKrFzF4WV-R0aUxeQ&oe=67061C71&_nc_sid=164c1d'),
 InstagramAccountInfo(name='Zelo', username='zeloloja', follower_count=243041, total_media=2975, last_update=datetime.date(2024, 10, 5), biography='Perfil oficial da Zelo, a mais completa rede especializada em cama, mesa e banho do Brasil.', profile_pic='https://scontent-atl3-1.cdninstagram.com/v/t51.2885-19/460159523_516877664613932_2282200909243022485_n.jpg?stp=dst-jpg_s640x640&_nc_ht=scontent-atl3-1.cdninstagram.com&_nc_cat=108&_nc_ohc=0Qmfnk7H_U0Q7kNvgGkJ7Uo&_nc_gid=984435272c6c425581eae5186e9194a1&edm=AO4kU9EBAAAA&ccb=7-5&oh=00_AYD9bxizUQ9IIF_R5SNf17F4xor6gGDAaUNIi1TaAKyAhw&oe=6705FE6B&_nc_sid=164c1d')
]

manager = InstagramDataManager()
service = InstagramMonitorService(messenger=Messenger())

dataset = manager.load_current_account_history_dataset(source='bigquery', query=QUERY_INSTAGRAM_MONITOR)
result = manager.update_accounts_history_dataset(dataset=dataset, account_info_list=account_info_list)
result
# service.update_accounts_info(usernames=usernames, debug_mode=True)


/Users/administrador/Library/Caches/pypoetry/virtualenvs/innovation-api-kUUtbMIs-py3.10/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(
/Users/administrador/Desktop/ammo/innovation-api/src/commom/instagram_data/instagram_data_manager.py:57: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_empty = pd.concat([df_empty, df_account_info]).reset_index(drop=True)


,name,username,follower_count,total_media,last_update,biography,profile_pic
0,Zelo,zeloloja,243041.0,2975.0,2024-10-05,"Perfil oficial da Zelo, a mais completa rede e...",https://scontent-atl3-1.cdninstagram.com/v/t51...
1,Trussardi,trussardioficial,294397.0,637.0,2024-10-05,Dal 1898.\nEstado da arte na rotina\nConheças ...,https://scontent-fra5-1.cdninstagram.com/v/t51...
2,Trousseau Oficial,trousseauoficial,256535.0,3838.0,2024-10-05,"Desde 1991 no universo de lifestyle, com produ...",https://scontent-ams4-1.cdninstagram.com/v/t51...
3,Santista,santistadecora,1405642.0,4417.0,2024-10-05,"Seu lar, nosso lugar 💙",https://scontent-fra5-2.cdninstagram.com/v/t51...
4,mmartan,mmartanoficial,1172367.0,4621.0,2024-10-05,"Desde 1980 criando histórias com a sua casa, s...",https://scontent-ams4-1.cdninstagram.com/v/t51...
5,Karsten Oficial,karstenoficial,737290.0,4231.0,2024-10-05,Casa renovada casa com você \nHá 141 anos traz...,https://scontent-mad2-1.cdninstagram.com/v/t51...
6,Hoomy,hoomybr,60576.0,483.0,2024-10-05,O único Lençol do Brasil feito para você dormi...,https://scontent-ams2-1.cdninstagram.com/v/t51...
7,Casa Riachuelo,casariachuelo,1110654.0,2213.0,2024-10-05,dicas e inspirações para você viver a sua casa...,https://scontent-atl3-2.cdninstagram.com/v/t51...
8,Casa Almeida,casaalmeidaoficial,103762.0,1545.0,2024-10-05,Uma experiência inigualável de sensações e est...,https://scontent-ams4-1.cdninstagram.com/v/t51...
9,Casa Sonno,casa.sonno,8894.0,201.0,2024-10-05,Roupas de cama para sonhadores ✸\nFeitas no Br...,https://scontent-lhr6-2.cdninstagram.com/v/t51...


## Nova feature

- Objetivo:
  - Adicionar a coluna delta

Regras da coluna delta

Devemos buscar os seguidores de cada conta, 
separar por data 
e fazer a diferença entre a maior e menor data


- Salvar localmente e no BQ
- Enviar dados com a coluna delta

In [128]:
from commom.data_classes.instagram_data_class import InstagramAccountInfo
from innovation_messenger import IEmailProperties, Messenger
from commom.database.queries.query_instagram_monitor import QUERY_INSTAGRAM_MONITOR
import pandas as pd
import datetime
from commom.instagram_data.instagram_data_manager import InstagramDataManager
from innovation_api.api.services.instagram_monitor_service import InstagramMonitorService

manager = InstagramDataManager()

dataset = manager.load_current_account_history_dataset(source='local', file_path='accounts_info.pkl')

In [129]:
import pandas as pd
import datetime

dataset['last_update'] = pd.to_datetime(dataset['last_update'])
tracked_usernames = dataset['username'].unique()
today = datetime.datetime.today().date()
month_init = datetime.datetime(today.year, today.month, 1)

dataset['delta_bruto'] = 0
dataset['delta_porcentagem'] = 0

for username in tracked_usernames:
    init_month_mask = dataset['last_update'] > month_init
    mask = dataset['username'] == username

    df_current_month: pd.DataFrame = dataset[mask & init_month_mask]

    if df_current_month.empty:
        print('vazio return delta 0')

    initial_count_mask = df_current_month['last_update'] == df_current_month['last_update'].min()
    initial_followers = df_current_month[initial_count_mask]['follower_count'].values[0]
    df_current_month['delta_bruto'] = df_current_month['follower_count'] - initial_followers
    df_current_month['delta_porcentagem'] = df_current_month['delta_bruto'] * 100.0 / initial_followers
    dataset[mask & init_month_mask] = df_current_month
dataset


TypeError: Invalid comparison between dtype=datetime64[us, UTC] and datetime

In [123]:
manager.save_current_account_history_dataset(source='bigquery', dataset_id='innovation_dataset', table_id='instagram_accounts_history', dataframe=dataset)

[INNOVATION-LOG] [write_to_bigquery] [Line (29)] [INFO]: Connection to BigQuery successful
[INNOVATION-LOG] [write_to_bigquery] [Line (39)] [INFO]: write done


True

In [132]:
from commom.data_classes.instagram_data_class import InstagramAccountInfo
from innovation_messenger import IEmailProperties, Messenger
from commom.database.queries.query_instagram_monitor import QUERY_INSTAGRAM_MONITOR
import pandas as pd
import datetime
from commom.instagram_data.instagram_data_manager import InstagramDataManager
from innovation_api.api.services.instagram_monitor_service import InstagramMonitorService

manager = InstagramDataManager()
columns = ['last_update', 'username','name','follower_count','delta_bruto', 'delta_porcentagem']
dataset = manager.load_current_account_history_dataset(source='bigquery', query=QUERY_INSTAGRAM_MONITOR)


/Users/administrador/Library/Caches/pypoetry/virtualenvs/innovation-api-kUUtbMIs-py3.10/lib/python3.10/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [134]:
dataset[columns].sort_values(by=['username','last_update'], ascending=False)

,last_update,username,name,follower_count,delta_bruto,delta_porcentagem
0,2024-10-07 00:00:00+00:00,zeloloja,Zelo,243061.0,49,0.020164
1,2024-10-05 00:00:00+00:00,zeloloja,Zelo,243041.0,29,0.011934
2,2024-10-04 00:00:00+00:00,zeloloja,Zelo,243041.0,29,0.011934
3,2024-10-03 00:00:00+00:00,zeloloja,Zelo,243012.0,0,0.000000
20,2024-10-07 00:00:00+00:00,trussardioficial,Trussardi,294659.0,352,0.119603
21,2024-10-05 00:00:00+00:00,trussardioficial,Trussardi,294397.0,90,0.030580
22,2024-10-04 00:00:00+00:00,trussardioficial,Trussardi,294397.0,90,0.030580
23,2024-10-03 00:00:00+00:00,trussardioficial,Trussardi,294307.0,0,0.000000
52,2024-10-07 00:00:00+00:00,trousseauoficial,Trousseau Oficial,256537.0,44,0.017154
53,2024-10-05 00:00:00+00:00,trousseauoficial,Trousseau Oficial,256535.0,42,0.016375
